# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fa544455370>
├── 'a' --> tensor([[ 0.6980,  0.6556, -1.8320],
│                   [ 0.2526,  0.6751,  0.9084]])
└── 'x' --> <FastTreeValue 0x7fa5444551c0>
    └── 'c' --> tensor([[ 1.5934, -0.4747, -1.5555,  0.8006],
                        [ 0.5814, -0.5241, -1.1314,  0.4969],
                        [-1.1678, -0.3324,  0.3408, -0.3150]])

In [4]:
t.a

tensor([[ 0.6980,  0.6556, -1.8320],
        [ 0.2526,  0.6751,  0.9084]])

In [5]:
%timeit t.a

72.1 ns ± 1.05 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fa544455370>
├── 'a' --> tensor([[-1.3721,  1.4971,  0.1597],
│                   [-2.1390,  1.6198, -1.6277]])
└── 'x' --> <FastTreeValue 0x7fa5444551c0>
    └── 'c' --> tensor([[ 1.5934, -0.4747, -1.5555,  0.8006],
                        [ 0.5814, -0.5241, -1.1314,  0.4969],
                        [-1.1678, -0.3324,  0.3408, -0.3150]])

In [7]:
%timeit t.a = new_value

78.6 ns ± 1.39 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.6980,  0.6556, -1.8320],
               [ 0.2526,  0.6751,  0.9084]]),
    x: Batch(
           c: tensor([[ 1.5934, -0.4747, -1.5555,  0.8006],
                      [ 0.5814, -0.5241, -1.1314,  0.4969],
                      [-1.1678, -0.3324,  0.3408, -0.3150]]),
       ),
)

In [10]:
b.a

tensor([[ 0.6980,  0.6556, -1.8320],
        [ 0.2526,  0.6751,  0.9084]])

In [11]:
%timeit b.a

79.5 ns ± 1.02 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.9394,  1.7141,  0.0675],
               [-1.1571,  0.5437, -0.5925]]),
    x: Batch(
           c: tensor([[ 1.5934, -0.4747, -1.5555,  0.8006],
                      [ 0.5814, -0.5241, -1.1314,  0.4969],
                      [-1.1678, -0.3324,  0.3408, -0.3150]]),
       ),
)

In [13]:
%timeit b.a = new_value

663 ns ± 7.24 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.03 µs ± 15.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.7 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

282 µs ± 4.22 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

263 µs ± 5.62 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fa48c709880>
├── 'a' --> tensor([[[ 0.6980,  0.6556, -1.8320],
│                    [ 0.2526,  0.6751,  0.9084]],
│           
│                   [[ 0.6980,  0.6556, -1.8320],
│                    [ 0.2526,  0.6751,  0.9084]],
│           
│                   [[ 0.6980,  0.6556, -1.8320],
│                    [ 0.2526,  0.6751,  0.9084]],
│           
│                   [[ 0.6980,  0.6556, -1.8320],
│                    [ 0.2526,  0.6751,  0.9084]],
│           
│                   [[ 0.6980,  0.6556, -1.8320],
│                    [ 0.2526,  0.6751,  0.9084]],
│           
│                   [[ 0.6980,  0.6556, -1.8320],
│                    [ 0.2526,  0.6751,  0.9084]],
│           
│                   [[ 0.6980,  0.6556, -1.8320],
│                    [ 0.2526,  0.6751,  0.9084]],
│           
│                   [[ 0.6980,  0.6556, -1.8320],
│                    [ 0.2526,  0.6751,  0.9084]]])
└── 'x' --> <FastTreeValue 0x7fa5464e5c10>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

45.5 µs ± 1.01 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fa48c6ef3a0>
├── 'a' --> tensor([[ 0.6980,  0.6556, -1.8320],
│                   [ 0.2526,  0.6751,  0.9084],
│                   [ 0.6980,  0.6556, -1.8320],
│                   [ 0.2526,  0.6751,  0.9084],
│                   [ 0.6980,  0.6556, -1.8320],
│                   [ 0.2526,  0.6751,  0.9084],
│                   [ 0.6980,  0.6556, -1.8320],
│                   [ 0.2526,  0.6751,  0.9084],
│                   [ 0.6980,  0.6556, -1.8320],
│                   [ 0.2526,  0.6751,  0.9084],
│                   [ 0.6980,  0.6556, -1.8320],
│                   [ 0.2526,  0.6751,  0.9084],
│                   [ 0.6980,  0.6556, -1.8320],
│                   [ 0.2526,  0.6751,  0.9084],
│                   [ 0.6980,  0.6556, -1.8320],
│                   [ 0.2526,  0.6751,  0.9084]])
└── 'x' --> <FastTreeValue 0x7fa48c6ef2b0>
    └── 'c' --> tensor([[ 1.5934, -0.4747, -1.5555,  0.8006],
                        [ 0.5814, -0.5241, -1.1314,  0.4969],
                 

In [23]:
%timeit t_cat(trees)

41.3 µs ± 1.19 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

80.1 µs ± 1.48 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.6980,  0.6556, -1.8320],
                [ 0.2526,  0.6751,  0.9084]],
       
               [[ 0.6980,  0.6556, -1.8320],
                [ 0.2526,  0.6751,  0.9084]],
       
               [[ 0.6980,  0.6556, -1.8320],
                [ 0.2526,  0.6751,  0.9084]],
       
               [[ 0.6980,  0.6556, -1.8320],
                [ 0.2526,  0.6751,  0.9084]],
       
               [[ 0.6980,  0.6556, -1.8320],
                [ 0.2526,  0.6751,  0.9084]],
       
               [[ 0.6980,  0.6556, -1.8320],
                [ 0.2526,  0.6751,  0.9084]],
       
               [[ 0.6980,  0.6556, -1.8320],
                [ 0.2526,  0.6751,  0.9084]],
       
               [[ 0.6980,  0.6556, -1.8320],
                [ 0.2526,  0.6751,  0.9084]]]),
    x: Batch(
           c: tensor([[[ 1.5934, -0.4747, -1.5555,  0.8006],
                       [ 0.5814, -0.5241, -1.1314,  0.4969],
                       [-1.1678, -0.3324,  0.3408, -0.3150]],
         

In [26]:
%timeit Batch.stack(batches)

103 µs ± 1.51 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.6980,  0.6556, -1.8320],
               [ 0.2526,  0.6751,  0.9084],
               [ 0.6980,  0.6556, -1.8320],
               [ 0.2526,  0.6751,  0.9084],
               [ 0.6980,  0.6556, -1.8320],
               [ 0.2526,  0.6751,  0.9084],
               [ 0.6980,  0.6556, -1.8320],
               [ 0.2526,  0.6751,  0.9084],
               [ 0.6980,  0.6556, -1.8320],
               [ 0.2526,  0.6751,  0.9084],
               [ 0.6980,  0.6556, -1.8320],
               [ 0.2526,  0.6751,  0.9084],
               [ 0.6980,  0.6556, -1.8320],
               [ 0.2526,  0.6751,  0.9084],
               [ 0.6980,  0.6556, -1.8320],
               [ 0.2526,  0.6751,  0.9084]]),
    x: Batch(
           c: tensor([[ 1.5934, -0.4747, -1.5555,  0.8006],
                      [ 0.5814, -0.5241, -1.1314,  0.4969],
                      [-1.1678, -0.3324,  0.3408, -0.3150],
                      [ 1.5934, -0.4747, -1.5555,  0.8006],
                      [ 0.5814, -

In [28]:
%timeit Batch.cat(batches)

190 µs ± 9.06 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

597 µs ± 16.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
